In [ ]:
import xml
import re

In [ ]:
srource_doc = xml.dom.minidom.parse("icon_sprites_src.svg")
src_svg = srource_doc.getElementsByTagName('svg')[0]


In [ ]:
target_doc = xml.dom.minidom.Document()
trgt_template_el = target_doc.createElement('template')
trgt_svg = target_doc.createElement('svg')
target_doc.appendChild(trgt_template_el)
trgt_template_el.appendChild(trgt_svg)

In [ ]:
for (name, value) in src_svg.attributes.items():
  if name in ['xmlns', 'xmlns:svg']:
    trgt_svg.setAttribute(name, value)

In [ ]:
# find the main layer in the source SVG
main_group = None
for grp in src_svg.getElementsByTagName('g'):
  if (grp.getAttribute('id') ==  'layer1'):
    main_group = grp

In [ ]:
# regexp to find nubmer in format "##.####" but be able to rpund them to two digints after decimal
num_pattern = '(\-?\d+)((\.\d+))?'

def round_two_digits(matchobj): 
  int_part = matchobj.group(1)
  frac_part = matchobj.group(3)
  if frac_part == None:
    return int_part
  else:
    x = round(float((f"{int_part}{frac_part}")),2)
    return f"{x}"


In [ ]:
# loop through all groups in the main layer
for grp in main_group.getElementsByTagName('g'):

    trgt_symbol_el = target_doc.createElement('symbol')
    trgt_symbol_el.setAttribute('id', grp.getAttribute('id'))
    trgt_svg.appendChild(trgt_symbol_el)

    # loop trough all children of a single group
    for child in grp.childNodes:
      if child.nodeName != '#text':

        style = child.getAttribute('style')
        name = child.nodeName

        # first empty rectangle is the vievport boudary
        if (name == 'rect' and style.startswith('fill:none')):
          viewBoxParams = [child.getAttribute(att_id) for att_id in ['x', 'y', 'width', 'height']]
          trgt_symbol_el.setAttribute('viewBox', " ".join(viewBoxParams))
        # otherwise copy the node over - it's jsut part of the icon design
        else:
          element = target_doc.createElement(name)
          # get source params
          elementParams = {k : v for k, v in child.attributes.items() if k not in ['style', 'd']}
          elementStyle = child.getAttribute('style')
          elementClass = 'svg-icon-accent' if (re.search("fill:#ff0000", style) != None) else 'svg-icon-main'
          # set targer params
          element.setAttribute('class',elementClass)
          if (name == 'path'):
            elementPath = child.getAttribute('d')
            newElementPath = re.sub(num_pattern, round_two_digits, elementPath)
            element.setAttribute('d',newElementPath)
          [element.setAttribute(k, v) for k,v in elementParams.items()]
          # append new element
          trgt_symbol_el.appendChild(element)

In [ ]:
style_string ="""
  .svg-icon-main {fill: var(--color); stroke: none;}
  .svg-icon-accent {fill: var(--color2); stroke: none;}
"""

trgt_style = target_doc.createElement('style')
trgt_style_txt = target_doc.createTextNode(style_string)
trgt_style.appendChild(trgt_style_txt)

In [ ]:
with open('IconSprites.vue', 'w') as f:
  xml_file_content = target_doc.toprettyxml(indent='  ')
  f.writelines([f"{l}\n" for l in xml_file_content.split('\n') if not l.startswith('<?xml')])
  f.write(trgt_style.toprettyxml(indent='  '))